In [ ]:
from pcbnew import *
board = LoadBoard("./kits_prettyvis.kicad_pcb")

In [ ]:
pctl = PLOT_CONTROLLER(board)
popt = pctl.GetPlotOptions()
popt.SetOutputDirectory("prettyvis")

popt.SetPlotFrameRef(False)
popt.SetLineWidth(FromMM(3.))

popt.SetAutoScale(False)
popt.SetScale(1)
popt.SetMirror(False)
popt.SetUseGerberAttributes(True)
popt.SetExcludeEdgeLayer(False);
popt.SetScale(5)
popt.SetUseAuxOrigin(False)

In [ ]:
for i,t in enumerate(board.GetTracks()):
    pctl.SetLayer(F_Cu)
    popt.SetMirror(False)
    pctl.OpenPlotfile("all_front_track_%05d"%i, PLOT_FORMAT_SVG, "Pretty!")
    pctl.PlotLayer()
    pctl.SetLayer(B_Cu)
    popt.SetMirror(True)
    pctl.OpenPlotfile("all_back_track_%05d"%i, PLOT_FORMAT_SVG, "Pretty!")
    pctl.PlotLayer()
    t.DeleteStructure()
for i,m in enumerate(board.GetModules()):
    pctl.SetLayer(F_Cu)
    popt.SetMirror(False)
    pctl.OpenPlotfile("all_front_module_%05d"%i, PLOT_FORMAT_SVG, "Pretty!")
    pctl.PlotLayer()
    pctl.SetLayer(B_Cu)
    popt.SetMirror(True)
    pctl.OpenPlotfile("all_back_module_%05d"%i, PLOT_FORMAT_SVG, "Pretty!")
    pctl.PlotLayer()
    m.DeleteStructure()
    c = i
pctl.SetLayer(F_Cu)
popt.SetMirror(False)
pctl.OpenPlotfile("all_front_module_%05d"%(c+1), PLOT_FORMAT_SVG, "Pretty!")
pctl.PlotLayer()
pctl.SetLayer(B_Cu)
popt.SetMirror(True)
pctl.OpenPlotfile("all_back_module_%05d"%(c+1), PLOT_FORMAT_SVG, "Pretty!")
pctl.PlotLayer()
pctl.ClosePlot()

In [ ]:
import glob
import os

In [ ]:
allfiles = sorted(glob.glob("prettyvis/kits_prettyvis-all_back_module_*"),reverse=True)
allfiles += sorted(glob.glob("prettyvis/kits_prettyvis-all_back_track_*"),reverse=True)
for i,f in enumerate(allfiles):
    os.system("mv %s prettyvis/back_%05d.svg"%(f,i))

In [ ]:
allfiles = sorted(glob.glob("prettyvis/kits_prettyvis-all_front_module_*"),reverse=True)
allfiles += sorted(glob.glob("prettyvis/kits_prettyvis-all_front_track_*"),reverse=True)
for i,f in enumerate(allfiles):
    os.system("mv %s prettyvis/front_%05d.svg"%(f,i))

In [ ]:
for i in range(len(allfiles)):
    os.system("inkscape -z -e prettyvis/front_%05d.png --export-area-drawing --export-dpi=300 prettyvis/front_%05d.svg"%(i,i))
    os.system("inkscape -z -e prettyvis/back_%05d.png --export-area-drawing --export-dpi=300 prettyvis/back_%05d.svg"%(i,i))

In [ ]:
for i in range(len(allfiles)):
    os.system("mogrify -fuzz 30%% -fill purple -opaque black prettyvis/back_%05d.png"%(i))
    os.system("mogrify -fuzz 30%% -fill purple -opaque black prettyvis/front_%05d.png"%(i))

In [ ]:
for i in range(len(allfiles)):
    os.system("montage prettyvis/*_%05d.png[1180x1180] -geometry +40+40 -tile 2x1 prettyvis/mon_%05d.png"%(i,i))

In [ ]:
os.system("mkdir prettyvis/tmp")
os.system("mv prettyvis/back* prettyvis/front* prettyvis/tmp")
os.system("mkdir prettyvis/slow")
os.system("mkdir prettyvis/fast")
for i in range(57+1):
    os.system("mv prettyvis/mon_%05d.png prettyvis/slow"%i)
os.system("mv prettyvis/mon_* prettyvis/fast")

In [ ]:
# !ffmpeg -y -framerate 15 -pattern_type glob -i "prettyvis/slow/mon*.png" -c:v libx264 -r 30 -pix_fmt yuv420p prettyvis/1.mp4

In [ ]:
# !ffmpeg -y -framerate 250 -pattern_type glob -i "prettyvis/fast/mon*.png" -c:v libx264 -r 30 -pix_fmt yuv420p prettyvis/2.mp4

# In case inkscape crashes on a few

In [ ]:
allns = {g.split("_")[-1].split(".")[0]
         for g in glob.glob("prettyvis/tmp/front*svg")}
backns = {g.split("_")[-1].split(".")[0]
          for g in glob.glob("prettyvis/tmp/back*png")}
backns = allns - backns
frontns = {g.split("_")[-1].split(".")[0]
           for g in glob.glob("prettyvis/tmp/front*png")}
frontns = allns - frontns
allbadns = set.union(frontns,backns)

In [ ]:
for n in allbadns:
    os.system("inkscape -z -e prettyvis/tmp/front_%s.png --export-area-drawing --export-dpi=300 prettyvis/tmp/front_%s.svg"%(n,n))
    os.system("inkscape -z -e prettyvis/tmp/back_%s.png --export-area-drawing --export-dpi=300 prettyvis/tmp/back_%s.svg"%(n,n))

In [ ]:
for n in allbadns:
    os.system("mogrify -fuzz 30%% -fill purple -opaque black prettyvis/tmp/back_%s.png"%(n))
    os.system("mogrify -fuzz 30%% -fill purple -opaque black prettyvis/tmp/front_%s.png"%(n))

In [ ]:
for n in allbadns:
    os.system("montage prettyvis/tmp/*_%s.png[1180x1180] -geometry +40+40 -tile 2x1 prettyvis/mon_%s.png"%(n,n))

In [ ]:
os.system("mv prettyvis/mon_* prettyvis/fast")

# Drafts

In [ ]:
modules = board.GetModules()
back_modules = [m for m in modules if m.GetLayerName()=="B.Cu"]
front_modules = [m for m in modules if m.GetLayerName()=="F.Cu"]

In [ ]:
tracks = board.GetTracks()
back_tracks = [t for t in tracks if t.GetLayerName()=="B.Cu" or t.GetClass()=="VIA"]
front_tracks = [t for t in tracks if t.GetLayerName()=="F.Cu" or t.GetClass()=="VIA"]

In [ ]:
popt.SetMirror(True)
pctl.SetLayer(B_Cu)
for i,t in enumerate(back_tracks):
    pctl.OpenPlotfile("back_track_%05d"%i, PLOT_FORMAT_SVG, "Pretty!")
    t.DeleteStructure()
    pctl.PlotLayer()
for i,m in enumerate(back_modules):
    pctl.OpenPlotfile("back_module_%05d"%i, PLOT_FORMAT_SVG, "Pretty!")
    m.DeleteStructure()
    pctl.PlotLayer()

In [ ]:
pctl.ClosePlot()

In [ ]:
pctl.SetLayer(F_Cu)
for i,t in enumerate(front_tracks):
    pctl.OpenPlotfile("front_track_%05d"%i, PLOT_FORMAT_SVG, "Pretty!")
    t.DeleteStructure()
    pctl.PlotLayer()
for i,m in enumerate(front_modules):
    pctl.OpenPlotfile("front_module_%05d"%i, PLOT_FORMAT_SVG, "Pretty!")
    m.DeleteStructure()
    pctl.PlotLayer()

In [ ]:
pctl.ClosePlot()